In [ ]:
from dcs.models.np_classifier import NPClassifier

NPClassifier()

In [ ]:
from dcs.models import NPClassifier, BBB

BBB().predict_from_csv("test.csv", "smiles", "ID")

In [6]:
from dcs.models import PlantsSMPrecursorPredictor, MixedPredictor, PlantsSMPrecursorPredictor, NPClassifier
MixedPredictor([PlantsSMPrecursorPredictor(), NPClassifier()]).predict_from_csv("test.csv", "smiles", "ID")

Predicting using Plants secondary metabolite precursors predictor model
Models already in cache.


LayeredFingerprint: 100%|██████████| 3/3 [00:00<00:00, 40.46it/s]

Predicting using NPClassifier model
Models already in cache.



2025-01-14 16:11:49.651983: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://194ce3e42b5245f19ee2bc9147559e6f: INVALID_ARGUMENT: ram://194ce3e42b5245f19ee2bc9147559e6f is a directory.
2025-01-14 16:12:05.676914: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://87223e884a1b4f909d5c96da176281e1: INVALID_ARGUMENT: ram://87223e884a1b4f909d5c96da176281e1 is a directory.
NPClassifierFP: 100%|██████████| 3/3 [00:00<00:00, 57.73it/s]

2025-01-14 16:12:06,033 — ERROR — 'Functional' object has no attribute 'predict_proba'
2025-01-14 16:12:06,035 — INFO — <keras.src.engine.functional.Functional object at 0x7fcad1c884c0>
2025-01-14 16:12:06,036 — INFO — <class 'keras.src.engine.functional.Functional'>


1/1 [==============================] - 0s 284ms/step


,ID,SMILES,Precursors,Pathways,Superclass,Class
0,cannabigerolic acid,CCCCCC1=CC(=C(C(=C1C(=O)O)O)C/C=C(\C)/CCC=C(C)C)O,,"Polyketides, Terpenoids",Meroterpenoids,Cannabinoids
1,cannabigerolic acid monomethylether,CCCCCC1=CC(=C(C(=C1C(=O)O)O)C/C=C(\C)/CCC=C(C)...,,"Polyketides, Terpenoids",Meroterpenoids,Cannabinoids
2,cannabigerol,CCCCCC1=CC(=C(C(=C1)O)C/C=C(\C)/CCC=C(C)C)O,,"Polyketides, Terpenoids",Meroterpenoids,Cannabinoids
3,test_invalid,CCC1=C(C)CN(C(=O)NCCc2ccc(S(=O$)(=O)NC(=O)N[C@...,,,,


In [ ]:
from deepmol.pipeline import VotingPipeline

pipeline = VotingPipeline.load("retrain/deployed_models/BBB_Martins/")

In [1]:
import torch 

torch.cuda.is_available()

True

In [ ]:


import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import tensorflow as tf
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

from dcs.models import BBB, PPBR, VDss, Caco2, HIA, Bioavailability, \
    Lipophilicity, Solubility, CYP2D6Inhibition, CYP2C9Inhibition, \
        CYP3A4Inhibition, CYP2C9Substrate, CYP3A4Substrate, CYP2D6Substrate, HepatocyteClearance, MicrosomeClearance, hERGBlockers, LD50, MixedPredictor, PlantsSMPrecursorPredictor, NPClassifier

# results = MixedPredictor([BBB(), Caco2(), CYP2D6Inhibition(), NPClassifier()]).predict_from_csv("test_molecules.csv", "Drug", "Drug_ID", output_file="predictions.csv")
results = MixedPredictor([BBB(), PPBR(), VDss(), Caco2(), 
                          HIA(), Bioavailability(), Lipophilicity(),
                          Solubility(), CYP2D6Inhibition(), CYP2C9Inhibition(), 
                          CYP3A4Inhibition(), CYP2C9Substrate(), CYP3A4Substrate(), CYP2D6Substrate(), 
                          HepatocyteClearance(), MicrosomeClearance(), hERGBlockers(), LD50()]).predict_from_csv("test.csv", "smiles", "ID", output_file="predictions.csv")
results

In [ ]:
results = results.dropna()
results

In [ ]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def generate_tsne_molecular_similarities(dataset, smiles_field):
    # Create a function to compute molecular fingerprints
    def compute_fingerprint(smiles):
        molecule = Chem.MolFromSmiles(smiles)
        fingerprint = AllChem.GetMorganFingerprintAsBitVect(molecule, 2, nBits=1024)
        return fingerprint

    # Compute molecular fingerprints for each molecule in the combined dataset
    fingerprints = [compute_fingerprint(smiles) for smiles in dataset.loc[:, smiles_field]]

    similarity_matrix = np.zeros((len(fingerprints), len(fingerprints)))
    # Compute molecular similarities using Tanimoto coefficient
    for i in range(len(fingerprints)):
        for j in range(i + 1, len(fingerprints)):
            similarity = DataStructs.TanimotoSimilarity(fingerprints[i], fingerprints[j])
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity

    # Apply t-SNE to reduce the dimensionality
    tsne = TSNE(n_components=2, random_state=42)
    tsne_embeddings = tsne.fit_transform(similarity_matrix)
    # Separate the embeddings based on the original datasets
    dataset["t-sne 1"] = tsne_embeddings[:, 0]
    dataset["t-sne 2"] = tsne_embeddings[:, 1]

    return dataset

dataset = generate_tsne_molecular_similarities(results, "SMILES")

In [ ]:
dataset

In [ ]:
import bokehmol
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral10

# label = "LD50_(log(1/(mol/kg)))"
label = "Human_PPBR"

# Register the dataset with BokehMol
bokehmol.register_alias()

def assign_color(value):
    if value == 1:
        return "green"
    else:
        return "orange"
    
y = dataset.loc[:, label]
    
color_mapper = linear_cmap(field_name=label, palette=Spectral10, low=min(y), high=max(y))

# Apply the function to create a color column
# dataset["color"] = dataset["Bioavailability"].apply(assign_color)

# Create ColumnDataSource
dataset.columns = [column.replace(" ", "_") for column in dataset.columns]
label = label.replace(" ", "_")
source = ColumnDataSource(dataset)

# Create plot
plot = figure(
    width=1000, height=1000,
    title="DeepMol Hover",
    background_fill_color="#efefef",
    tools="rdkit_hover,pan,wheel_zoom",
)

mol_hover = bokehmol.hover.rdkit(
    smiles_column="SMILES",
    tooltips=[
        ("ID", "@ID"),
        ("Precursors", "@Precursors"),
        ("Pathway", "@Pathways"),
        ("Superclass", "@Superclass"),
        ("Class", "@Class"),
        
    ],
    draw_options={
        "comicMode": True
    }
)

plot.add_tools(mol_hover)

# Add circles to plot with color mapping
# plot.circle(
#     "t-sne 1", "t-sne 2",
#     size=15, line_width=0,
#      legend_group="Bioavailability",
#     color="color",
#     source=source
# )

plot.circle(
    "t-sne_1", "t-sne_2",
    size=15, line_width=0,
    color=color_mapper, legend_label=label,
    source=source
)

from bokeh.models import ColumnDataSource, ColorBar, LinearColorMapper

color_bar = ColorBar(color_mapper=LinearColorMapper(palette=Spectral10, low=min(y), high=max(y)), width=8, location=(0,0))
plot.add_layout(color_bar, 'right')

# Show plot
show(plot)


In [ ]:
dataset